In [15]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from gensim.models import Word2Vec

In [16]:
g = nx.read_gpickle("github.gpickle.2")
#g = nx.read_gexf('les-miserables.gexf')
#nx.write_graphml(g, "github.graphml")

In [17]:
print nx.info(g)

Name: 
Type: DiGraph
Number of nodes: 96833
Number of edges: 233089
Average in degree:   2.4071
Average out degree:   2.4071


In [18]:
def add_w(g):
    for edge in g.edges():
        g[edge[0]][edge[1]]['weight'] = 1
    return g


In [19]:
D = True
P = 1
#Return hyperparameter. Default is 1
Q = 1
#Inout hyperparameter. Default is 1.
NW = 10
#Number of walks per source. Default is 40
WL = 40
#Length of walk per source. Default is 10.
WORKERS = 8
#'Number of parallel workers. Default is 8.
WIN_SIZE = 4
#Context size for optimization. Default is 10.
ITER = 20
#Number of epochs in SGD
DIM = 64
#Number of dimensions. Default is 128.



In [20]:
nx_G = add_w(g)

G = Graph(nx_G, D, P, Q)
G.preprocess_transition_probs()
walks = G.simulate_walks(NW, WL)
#learn_embeddings(walks)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [21]:
def learn_embeddings(walks):
    '''
    Learn embeddings by optimizing the Skipgram objective using SGD.
    '''
    walks = [map(str, walk) for walk in walks]
    model = Word2Vec(walks, size=DIM, window=WIN_SIZE, min_count=0, workers=WORKERS, iter=ITER)
    return model

model = learn_embeddings(walks)
model.save_word2vec_format("output2.emb")


In [22]:
%%time

from sklearn.cluster import KMeans

word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

# For the first 10 clusters
for cluster in xrange(0,20):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
['push_it(repo)', 'omwcat(repo)', 'sbt-groll(repo)']

Cluster 1
['Mining-the-Social-Web(repo)']

Cluster 2
['pparo(user)']

Cluster 3
['kohana-rest-framework(repo)', 'ocaml-eff-example(repo)', 'sublime-twitter-bootstrap-snippets(repo)', 'new_maker_press(repo)', 'clojure-ttt(repo)']

Cluster 4
['TV-kko(repo)', 'qsandra(repo)', 'IPython-Dashboard(repo)', 'genevalidatorapp(repo)']

Cluster 5
['Bohr(repo)', 'ditto(repo)', 'Freematics(repo)', 'tg2(repo)', 'dotnet(repo)', 'no-wang-wang(repo)', 'TKRubberIndicator(repo)', 'slsw_erlang(repo)']

Cluster 6
['wp-demo-plugin-i18n(repo)', 'binaryen(repo)', 'angularJS-TwitterSearch(repo)']

Cluster 7
['node-vue-server-webpack(repo)', 'CleverRecyclerView(repo)', 'hashID(repo)', 'nv-websocket-client(repo)', 'chrome-nico-hatebu(repo)', 'YoutubeAudioClipper(repo)', 'isbnnetinclj2(repo)', 'PokeMobBot(repo)']

Cluster 8
['todomvc-aurelia(repo)', 'locache(repo)']

Cluster 9
['flipcounter(repo)', 'pytest-blockage(repo)', 'mendicantuniversity.org(r

In [26]:
for cluster in xrange(20,100):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 20
['react-ratchet(repo)', 'TiFaceRecognizer(repo)', 'tmux-mem-cpu-load(repo)', 'ManageSieveJ(repo)', 'FastRoute(repo)', 'iv(repo)', 'osm-postgis-scripts(repo)', 'picasso2-okhttp3-downloader(repo)', 'ec2price(repo)']

Cluster 21
['Planet-Wars-Canvas-Visualizer(repo)', 'resume.pdf(repo)', 'earth-latest-data(repo)', 'Pokemon-Go-Rocket-API(repo)', 'spry(repo)']

Cluster 22
['JohnTheRipper(repo)', 'Java-Download-Manager(repo)', 'nutz(repo)', 'zen-and-art(repo)']

Cluster 23
['ptwobrussell(user)']

Cluster 24
['luci-access-control(repo)', 'javascript-libraries-syntax.vim(repo)', 'divingintoipynb(repo)', 'wheelhouse(repo)', 'lamson(repo)', 'formd(repo)', 'elixir_kata(repo)', 'MBrace.Core(repo)', 'gender(repo)']

Cluster 25
['getting_started_guide(repo)', 'python-phabricator(repo)', 'material-intro(repo)', 'django-adminplus(repo)', 'awesome-react(repo)', 'html-email-snippets(repo)', 'eventlet_visualiser(repo)', 'ssp(repo)']

Cluster 26
['nearley(repo)', 'AndroidDnssdDemo(repo)', 'pyc

In [23]:
#model.most_similar(positive=['d3(repo)', 'jquery(repo)'], negative=['EthereumContracts(repo)'])

In [24]:
#print model.similarity('d3(repo)', 'jquery(repo)')
#print model.similarity('Big_Data_Resources(repo)', 'jquery(repo)')

In [27]:
model.most_similar('Dumbris(user)')

KeyError: "word 'Dumbris(user)' not in vocabulary"

In [ ]:
sp = nx.all_shortest_paths(g,source='Dumbris(user)',target='titan(repo)')
print([p for p in sp])
#nx.astar_path(g,source='davni(user)',target='ekgren(user)')

In [ ]:
model.similar_by_vector(model['Dumbris(user)'] + model['jfenton(user)'])

In [ ]:
import random


class Graph():
	def __init__(self, nx_G, is_directed, p, q):
		self.G = nx_G
		self.is_directed = is_directed
		self.p = p
		self.q = q

	def node2vec_walk(self, walk_length, start_node):
		'''
		Simulate a random walk starting from start node.
		'''
		G = self.G
		alias_nodes = self.alias_nodes
		alias_edges = self.alias_edges

		walk = [start_node]

		while len(walk) < walk_length:
			cur = walk[-1]
			cur_nbrs = sorted(G.neighbors(cur))
			if len(cur_nbrs) > 0:
				if len(walk) == 1:
					walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
				else:
					prev = walk[-2]
					next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
						alias_edges[(prev, cur)][1])]
					walk.append(next)
			else:
				break

		return walk

	def simulate_walks(self, num_walks, walk_length):
		'''
		Repeatedly simulate random walks from each node.
		'''
		G = self.G
		walks = []
		nodes = list(G.nodes())
		print 'Walk iteration:'
		for walk_iter in range(num_walks):
			print str(walk_iter+1), '/', str(num_walks)
			random.shuffle(nodes)
			for node in nodes:
				walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

		return walks

	def get_alias_edge(self, src, dst):
		'''
		Get the alias edge setup lists for a given edge.
		'''
		G = self.G
		p = self.p
		q = self.q

		unnormalized_probs = []
		for dst_nbr in sorted(G.neighbors(dst)):
			if dst_nbr == src:
				unnormalized_probs.append(G[dst][dst_nbr]['weight']/p)
			elif G.has_edge(dst_nbr, src):
				unnormalized_probs.append(G[dst][dst_nbr]['weight'])
			else:
				unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
		norm_const = sum(unnormalized_probs)
		normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

		return alias_setup(normalized_probs)

	def preprocess_transition_probs(self):
		'''
		Preprocessing of transition probabilities for guiding the random walks.
		'''
		G = self.G
		is_directed = self.is_directed

		alias_nodes = {}
		for node in G.nodes():
			unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
			norm_const = sum(unnormalized_probs)
			normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
			alias_nodes[node] = alias_setup(normalized_probs)

		alias_edges = {}
		triads = {}

		if is_directed:
			for edge in G.edges():
				alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
		else:
			for edge in G.edges():
				alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
				alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])

		self.alias_nodes = alias_nodes
		self.alias_edges = alias_edges

		return


def alias_setup(probs):
	'''
	Compute utility lists for non-uniform sampling from discrete distributions.
	Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
	for details
	'''
	K = len(probs)
	q = np.zeros(K)
	J = np.zeros(K, dtype=np.int)

	smaller = []
	larger = []
	for kk, prob in enumerate(probs):
	    q[kk] = K*prob
	    if q[kk] < 1.0:
	        smaller.append(kk)
	    else:
	        larger.append(kk)

	while len(smaller) > 0 and len(larger) > 0:
	    small = smaller.pop()
	    large = larger.pop()

	    J[small] = large
	    q[large] = q[large] + q[small] - 1.0
	    if q[large] < 1.0:
	        smaller.append(large)
	    else:
	        larger.append(large)

	return J, q

def alias_draw(J, q):
	'''
	Draw sample from a non-uniform discrete distribution using alias sampling.
	'''
	K = len(J)

	kk = int(np.floor(np.random.rand()*K))
	if np.random.rand() < q[kk]:
	    return kk
	else:
	    return J[kk]
